## 1. 데이터 로딩

코모란으로 토크나이징이 되어있는 데이터를 로딩합니다.

In [1]:
from config import dataset_dir
import sys
sys.path.append(dataset_dir)

from lovit_textmining_dataset.navermovie_comments import load_movie_comments

idxs, texts, rates = load_movie_comments(large=False, tokenize='komoran')

## 2. Selecting training data and vectorizing

단어의 min frequency cutting 을 하기 위해 먼저 texts 에 있는 모든 단어들의 빈도수를 확인합니다. 

    [word for text in texts for word in text.split()]
    
위 list 는 texts 의 각 text 마다 split() 을 한 뒤, 그 단어들을 하나의 list 로 flatten 하는 코드입니다. 

    ['a b', 'c d e'] --> ['a', 'b', 'c', 'd', 'e']

처럼 결과가 나옵니다. 이를 list comprehension 이라 합니다.

In [2]:
from collections import Counter
word_counter = Counter([word for text in texts for word in text.split()])

빈도수 기준 상위 20개의 단어를 출력합니다. 

In [3]:
sorted(word_counter.items(), key=lambda x:x[1], reverse=True)[:20]

[('ㄴ/ETM', 108821),
 ('영화/NNG', 94681),
 ('보/VV', 90793),
 ('이/VCP', 88335),
 ('고/EC', 84500),
 ('하/XSV', 80346),
 ('다/EC', 79071),
 ('는/ETM', 75542),
 ('이/JKS', 73053),
 ('았/EP', 67376),
 ('에/JKB', 59791),
 ('었/EP', 58573),
 ('하/XSA', 56066),
 ('관람객/NNG', 52739),
 ('의/JKG', 48716),
 ('게/EC', 48522),
 ('ㄹ/ETM', 46382),
 ('가/JKS', 45865),
 ('도/JX', 43278),
 ('는/JX', 41974)]

최소 빈도수를 10 으로 설정합니다. 9 번 이하로 등장한 단어는 term frequency vector 에 이용하지 않습니다. 총 10,179 개의 단어가 포함되어 있습니다.

In [4]:
n_before = len(word_counter)

min_count = 10
word_dictionary = {
    word:freq for word,freq in word_counter.items()
    if freq >= min_count
}

n_after  = len(word_dictionary)

print('%d --> %d' % (n_before, n_after))

64330 --> 10179


평점 별로도 문서의 개수를 확인합니다. 유명한 영화들이다보니 10점이 압도적으로 많음을 알 수 있습니다. 애초에 좋은 영화가 아니면 사람들이 많이 보지 않았겠죠? 

In [5]:
for rate, freq in sorted(Counter(rates).items()):
    perc = 100 * freq / len(rates)
    print('rate = {}: ({}, {:.3} %)'.format(rate, freq, perc))

rate = 1: (56122, 19.1 %)
rate = 2: (4725, 1.6 %)
rate = 3: (4547, 1.54 %)
rate = 4: (4062, 1.38 %)
rate = 5: (7697, 2.61 %)
rate = 6: (7588, 2.58 %)
rate = 7: (11338, 3.85 %)
rate = 8: (20311, 6.9 %)
rate = 9: (25528, 8.67 %)
rate = 10: (152575, 51.8 %)


min count cutting 을 하다보니 희귀한 단어로만 이뤄진 문장도 존재할 수 있습니다. 이 경우에는 zero vector 가 됩니다. 이를 방지하기 위해서 nonzero vector 만으로 이뤄진 학습용 데이터를 만듭니다. text를 split() 한 다음, 각 단어가 word_dictionary (min_count >= 10인 단어 집합)에 등록되었는지 확인합니다. 만약 words가 empty list이면 학습데이터에 추가하지 않고 다음 text를 살펴봅니다. 

    words = [word for word in text.split() if word in word_dictionary]
    if not words:
        continue

그리고 binary classification 을 하기 위해서 3점 이하의 영화평을 negative class, 9점 이상의 영화평을 positive class 로 정의합니다. 이를 위하여 4 ~ 8 점 사이의 리뷰들은 모두 무시합니다.

    if 4 <= score <= 8:
        continue

점수가 8 보다 크면 positive 인 1 점을, 그렇지 않다면 negative 인 -1 점을 입력합니다.

    train_label.append(1 if score > 8 else -1)

여기서 한 가지 조금 특이하게 train_texts 를 만들었습니다. words 는 list of str입니다. 그렇기 때문에 train_texts 는 nested list 입니다. 

In [7]:
import numpy as np

train_texts = []
train_label = []

for text, rate in zip(texts, rates):

    # skip 4 ~ 8 scored reviews
    if 4 <= rate <= 8:
        continue

    # skip empty reviews
    words = [word for word in text.split() if word in word_dictionary]
    if not words:
        continue

    # append text and label
    train_texts.append(words)
    train_label.append(1 if rate > 8 else -1)
train_label = np.asarray(train_label)

print('train data: %d --> %d' % (len(texts), len(train_texts)))

for label, freq in Counter(train_label).items():
    perc = 100 * freq / len(train_label)
    print('label = {}: ({}, {:.3} %)'.format(label, freq, perc))

train data: 294493 --> 239708
label = 1: (175257, 73.1 %)
label = -1: (64451, 26.9 %)


CountVectorizer 를 이용하여 train_x, term frequency matrix 를 만듭니다. train_texts 는 이미 토크나이징이 완료되어 있습니다. CountVectorizer 에서 lowercase=False 로 만들고, tokenizer를 lambda x:x로 두면 아무런 처리를 하지 않은 체 train_texts 를 vectorizer 에 입력할 수 있습니다. 

sparse matrix 에서 이용한 단어의 개수가 10,179 개로 word_dictionary 의 개수와 같습니다. min_df, max_df 를 설정하지 않았기 때문에 빈도수가 10 이상인 모든 단어들을 이용하여 sparse matrix 를 만듭니다.

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(tokenizer=lambda x:x, lowercase=False)
train_x = vectorizer.fit_transform(train_texts)
train_x.shape

(239708, 10178)

이전의 실습처럼 vectorizer.vocabulary\_ 로부터 vocablist 를 만들 수 있습니다. 

In [9]:
idx_to_vocab = [
    word for word, _ in sorted(
        vectorizer.vocabulary_.items(), key=lambda x:x[1]
    )
]

In [10]:
idx_to_vocab[1000:1005]

['경외심/NNG', '경우/NNG', '경우/NNP', '경의/NNG', '경이/NNG']

여기까지하여 만들어둔 데이터를 data/ 폴더에 저장해 두었습니다. 

In [11]:
SAVE = False
if SAVE:
    import pickle
    directory = '{}/lovit_textmining_dataset/navermovie_comments/models/'.format(data_dir)    
    with open('{}/sentiment_small_komoran_x.pkl'.format(directory), 'wb') as f:
        pickle.dump(train_x, f)

    with open('{}/sentiment_small_komoran_y.pkl'.format(directory), 'wb') as f:
        pickle.dump(train_label, f)

    with open('{}/sentiment_small_komoran_vocab.txt'.format(directory), 'w', encoding='utf-8') as f:
        for vocab in idx_to_vocab:
            f.write('%s\n' % vocab)

    with open('{}/sentiment_small_komoran_texts.txt'.format(directory), 'w', encoding='utf-8') as f:
        for text in train_texts:
            f.write('%s\n' % ' '.join(text))

이 데이터를 로딩하는 부분도 함수로 만들어두면, 이후에 여기부터 시작할 수 있습니다.

In [26]:
import warnings
warnings.filterwarnings('ignore')

from lovit_textmining_dataset.navermovie_comments import load_sentiment_dataset

train_texts, train_x, train_label, idx_to_vocab = load_sentiment_dataset(model_name='small', tokenize='komoran')

## 3. Training Logistic Regression (L2 regularization)

In [27]:
from sklearn.linear_model import LogisticRegression

logistic_l2 = LogisticRegression(verbose=True)
logistic_l2.fit(train_x, train_label)

[LibLinear]

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=True, warm_start=False)

임의의 문장 [1, 17603, 64323] 에 대하여 실제 데이터와 레이블, 그리고 logistic regression 이 예측하는 class probability / class label 을 출력합니다. 

    logistic_l2.predict_proba(text_x)
    
Logistic Regression은 predict 의 input으로 matrix가 들어올 것을, 동시에 여러 개의 queries가 들어올 것을 가정하고 만든 알고리즘입니다. 하나의 query를 prediction 할 때에는 그 결과값을

    logistic_l2.predict_proba(train_x[idx,:])[0]
    
과 같이 하여야 됩니다. 

In [14]:
def check_samples(sample_idxs, model, marker='-'*50):
    for idx in sample_idxs:
        prob = model.predict_proba(train_x[idx,:])[0]
        print('class prob: (negative= {:.3}, positive= {:.3}'.format(*prob))

        pred_label = model.predict(train_x[idx,:])[0]
        sentiment = 'positive' if pred_label == 1 else 'negative'
        print('prediction = {}'.format(sentiment))
        print('label = {}'.format(train_label[idx]))
        print('text: {}'.format(' '.join(train_texts[idx])))
        print(marker, end='\n\n')

check_samples([1, 17603, 64323], logistic_l2)

class prob: (negative= 3.18e-05, positive= 1.0
prediction = positive
label = 1
text: 인셉션/NNP 정말/MAG 흥미진진/XR 하/XSA 게/EC 보/VV 았었/EP 고/EC 크리스토퍼/NNP 놀라/VV ㄴ/ETM 감독/NNG 님/XSN 신작/NNP 인터스텔라/NNP 도/JX 이번/NNG 주/NNP 일요일/NNG 에/JKB 보/VV 러/EC 가/VX ㅂ니다/EC 완전/NNG 기대/NNG 중/NNB
--------------------------------------------------

class prob: (negative= 0.05, positive= 0.95
prediction = positive
label = 1
text: 관람객/NNG Great/SL
--------------------------------------------------

class prob: (negative= 1.0, positive= 0.000228
prediction = negative
label = -1
text: 개/NNB 거지/NNG 같/VA 은/ETM 영화/NNP 보/VV ㄹ/ETM 가치/NNG 도/JX 없/VA 는/ETM 삼류/NNG 쓰레기/NNP 영화/NNP 임/NNP ㄹㅇ/NA 걍/NA 무서움/NNG 만/JX 조장/NNG 하/XSV 고/EC 많/VA 은/ETM 의미/NNG 부여/NNG 하/XSV 려다가/EC 결말/NNG 도/JX 이상/XR 하/XSA 게/EC 끝나/VV ㄴ듯/EC
--------------------------------------------------



## 4. Coefficients of Logistic Regression

학습된 Logistic Regression 의 Coefficients 는 아래에 위치합니다.

    LogisticRegression.coef_
    
형식은 (1, n_vocab) 크기의 numpy.ndarray 입니다. 우리가 binary classification 을 하였기 때문입니다. multi class logistic regression 하면 (n_class, n_vocab) 의 coefficient matrix 가 만들어집니다. 

학습하는 방법은 위와 동일합니다. LogisticRegression 이 train_label 의 unique label 의 개수를 확인한 뒤, multi-class classification 이면 알아서 Softmax regression 으로 바꿔둡니다. 

In [15]:
print(type(logistic_l2.coef_))
print(logistic_l2.coef_.shape)

<class 'numpy.ndarray'>
(1, 10178)


이 ndarray 를 list 로 변형합니다. (1, n_vocab) 이기 때문에 coefs[0] 는 positive class 를 예측하는 각 단어의 coef 가 출력됩니다. 

In [16]:
coefs = logistic_l2.coef_.tolist()[0]
coefs[:5]

[-1.8612359515672794,
 -0.9174402663117003,
 -0.049708301010527436,
 0.20347381741519238,
 0.24223533212987516]

Positive class 에 가까운 단어들을 살펴보겠습니다. enumerate 를 이용하면 (idx, coef) 가 만들어집니다. 이를 coefficient 의 값이 큰 순서대로 정렬합니다.

In [17]:
idx_coefs = enumerate(coefs)
positives = sorted(idx_coefs, key=lambda x:-x[1])
positives[:5]

[(5930, 4.500462141524582),
 (6897, 4.216322072204339),
 (6643, 3.098290358270354),
 (8879, 2.762886320829506),
 (9141, 2.735697159564822)]

긍정적인 영화평에서 자주 나오는 단어들 상위 50개를 출력하면 아래와 같습니다. 

In [18]:
for word, coef in positives[:50]:
    print('{} ({:.3})'.format(idx_to_vocab[word], coef))

알이즈웰/NA (4.5)
웰/NA (4.22)
완벽/NNP (3.1)
충무로/NNP (2.76)
태식이/NNP (2.74)
재밋는데/NA (2.55)
황홀/XR (2.54)
GOOD/SL (2.53)
완벽/NNG (2.51)
재밋던데/NA (2.47)
김래원/NNP (2.45)
파이팅/NNP (2.41)
압권/NNG (2.4)
감탄사/NNP (2.39)
틈/NNG (2.36)
소름/NNP (2.35)
여운/NNP (2.34)
태식/NNP (2.34)
강렬/XR (2.32)
우주여행/NNP (2.32)
이즈웰/NA (2.31)
good/SL (2.31)
압도/NNG (2.29)
만족/NNP (2.28)
김세윤/NNP (2.27)
최고/NNG (2.25)
떼/VV (2.25)
꿀/NNG (2.24)
굿/NNG (2.23)
찝찝하다고/NA (2.22)
대박/NNP (2.18)
유건/NNP (2.17)
굳/VV (2.16)
유쾌/XR (2.16)
고의/NNP (2.15)
최고/NNP (2.14)
화이팅/NNP (2.14)
well/SL (2.13)
넋/NNG (2.13)
찡하/VA (2.12)
박사/NNP (2.11)
빨리/VV (2.1)
만족/NNG (2.1)
타스/NNP (2.09)
전문가/NNP (2.09)
씨네21/NNP (2.08)
울컥/MAG (2.08)
테러/NNG (2.05)
홧팅/NA (2.05)
뭉클/XR (2.04)


부정적인 영화평에서 자주 나오는 단어들 상위 50개를 출력하면 아래와 같습니다. 

In [19]:
idx_coefs = enumerate(coefs)
negatives = sorted(idx_coefs, key=lambda x:x[1])

for word, coef in negatives[:50]:
    print('{} ({:.3})'.format(idx_to_vocab[word], coef))

최악/NNG (-3.96)
과대평가/NNG (-3.53)
노/NNG (-3.42)
집기/NNG (-3.2)
나쁜 (-3.09)
베를린/NNP (-3.07)
된장/NNP (-2.98)
긴급조치/NNP (-2.92)
실망/NNP (-2.89)
환불/NNG (-2.87)
단세포/NNG (-2.85)
재앙/NNG (-2.81)
쓰레기/NNP (-2.77)
퇴보/NNG (-2.76)
하품/NNP (-2.75)
빵점/NNG (-2.71)
난감/XR (-2.67)
차라리/MAG (-2.67)
원주율/NNP (-2.64)
동참/NNG (-2.59)
방어/NNG (-2.56)
비추이/VV (-2.55)
독일/NNP (-2.54)
거품/NNG (-2.52)
마케팅/NNP (-2.51)
불면증/NNP (-2.5)
망신/NNG (-2.49)
고지전/NNP (-2.48)
포장/NNP (-2.46)
숙면/NNG (-2.45)
으리/EC (-2.44)
우롱/NNG (-2.37)
피바다/NNP (-2.35)
실소/NNG (-2.33)
제로/NNP (-2.32)
오토바이/NNG (-2.32)
쓰렉/NA (-2.32)
거품/NNP (-2.31)
공짜/NNG (-2.3)
마이너스/NNP (-2.29)
노/NNP (-2.27)
실망/NNG (-2.26)
포장/NNG (-2.26)
불쾌감/NNG (-2.25)
안드로메다/NNP (-2.24)
오씨/NNG (-2.23)
평화/NNG (-2.22)
스크린쿼터/NNP (-2.21)
화나/VV (-2.2)
아메바/NNP (-2.2)


## 4. Logistic Regression with L1 regularization

Lasso Regression 은 penalty 설정만 바꿔주면 됩니다. 기본값은 'l2' 입니다. C 는 $\frac{1}{\lambda}$ 입니다. C 의 값이 작을수록 큰 regularity 가 걸립니다.

In [20]:
from sklearn.linear_model import LogisticRegression

logistic_l1 = LogisticRegression(penalty='l1', C=0.1)
logistic_l1.fit(train_x, train_label)

/home/lovit/anaconda2/envs/scrapper/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l1', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

Lasso Regression 의 결과도 확인합니다.

In [21]:
check_samples([1, 17603, 64323], logistic_l1)

class prob: (negative= 0.000216, positive= 1.0
prediction = positive
label = 1
text: 인셉션/NNP 정말/MAG 흥미진진/XR 하/XSA 게/EC 보/VV 았었/EP 고/EC 크리스토퍼/NNP 놀라/VV ㄴ/ETM 감독/NNG 님/XSN 신작/NNP 인터스텔라/NNP 도/JX 이번/NNG 주/NNP 일요일/NNG 에/JKB 보/VV 러/EC 가/VX ㅂ니다/EC 완전/NNG 기대/NNG 중/NNB
--------------------------------------------------

class prob: (negative= 0.138, positive= 0.862
prediction = positive
label = 1
text: 관람객/NNG Great/SL
--------------------------------------------------

class prob: (negative= 0.995, positive= 0.00511
prediction = negative
label = -1
text: 개/NNB 거지/NNG 같/VA 은/ETM 영화/NNP 보/VV ㄹ/ETM 가치/NNG 도/JX 없/VA 는/ETM 삼류/NNG 쓰레기/NNP 영화/NNP 임/NNP ㄹㅇ/NA 걍/NA 무서움/NNG 만/JX 조장/NNG 하/XSV 고/EC 많/VA 은/ETM 의미/NNG 부여/NNG 하/XSV 려다가/EC 결말/NNG 도/JX 이상/XR 하/XSA 게/EC 끝나/VV ㄴ듯/EC
--------------------------------------------------



L2 regression 은 모든 단어를 이용하여 classification 을 수행하지만, L1 regression 은 일부의 단어 (변수)만을 이용하여 classification 을 수행합니다.

In [22]:
import numpy as np

nnz_l1 = np.where(abs(logistic_l1.coef_) > 0)[0].shape
nnz_l2 = np.where(abs(logistic_l2.coef_) > 0)[0].shape

print('Number of nonzero (L1 logistic) = {}'.format(nnz_l1))
print('Number of nonzero (L2 logistic) = {}'.format(nnz_l2))

Number of nonzero (L1 logistic) = (1626,)
Number of nonzero (L2 logistic) = (10178,)


C 를 조절하면 Lasso Regression 에서 coefficient 가 0 인 단어의 개수가 달라집니다.

In [23]:
for C in [10, 1, 0.1]:
    logistic_l1_ = LogisticRegression(penalty='l1', C=C)
    logistic_l1_.fit(train_x, train_label)
    nnz = np.where(abs(logistic_l1_.coef_) > 0)[0].shape
    print('Cost = {}, nnz = {}'.format(C, nnz))

/home/lovit/anaconda2/envs/scrapper/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Cost = 10, nnz = (9690,)
Cost = 1, nnz = (6518,)
Cost = 0.1, nnz = (1626,)
